In [ ]:
import polars as pl
import json
import time
import os

QUERIES_FOLDER = "../1.query/dynamic_features/"
OUTPUT_FOLDER = '../2.rawDataset/dynamic_features/'

In [ ]:
encounters = pl.read_parquet('../4.preparedDatasets/clean_static_encounters.parquet').select("encounterId", "utcInTime")

In [ ]:
HOST = 'localhost'
DATABASE = 'mimiciv'
conn_uri = f'postgresql://postgres:password@{HOST}/{DATABASE}'

query = open(QUERIES_FOLDER + 'generic_query.sql').read()

In [ ]:
with open('../1.query/dynamic_features/' + 'features.json', 'r', encoding='utf-8') as f :
    variables_json = json.load(f)

## Génération des datasets

### Automatisée via JSON

In [ ]:
print(list(variables_json.keys()))

In [ ]:
variables_list = ['pam_non_invasive']
variable_extraction = list(variables_json.keys()) # {key: variables_json[key] for key in variables_list} / list(variables_json.keys())


In [ ]:
for variable in variable_extraction :
    print(f'---------Extraction {variable} : START ---------')

    loop_time = time.time()

    # Récupération des paramètres pour l'extraction depuis le json
    propname = variables_json[variable]['dictionary_propname']
    table = variables_json[variable]['table']
    feature = variables_json[variable]

    # Implémentation des paramètres dans la requête SQL
    query_formatted = query.format(
        dictionaryPropName = propname,
        feature = f"'{variable}'",
        feature_table = table
    )
print(query_formatted)

In [ ]:
pl.read_database_uri(query_formatted, conn_uri, engine='connectorx')

In [ ]:
start = time.time()
for variable in variable_extraction :
    print(f'---------Extraction {variable} : START ---------')

    loop_time = time.time()

    # Récupération des paramètres pour l'extraction depuis le json
    propname = variables_json[variable]['dictionary_propname']
    table = variables_json[variable]['table']
    feature = variables_json[variable]

    # Implémentation des paramètres dans la requête SQL
    query_formatted = query.format(
        dictionaryPropName = propname,
        feature = f"'{variable}'",
        feature_table = table
    )
    
    try :

        # Extraction des données et stockage dans un dataframe polars
        feature_data = pl.read_database_uri(query_formatted, conn_uri, engine='connectorx')

        print(f'Extraction {variable} : OK')

        # Récupération uniquement des encounterId contenu dans le dataset 'statique'
        with_time_delta = (feature_data
        .rename(
           { 'encounterid': 'encounterId',
            'utccharttime': 'utcChartTime',
            'valuestring': 'valueString',
            'valuenumber': 'valueNumber'
            }
        )
        .with_columns(
            pl.col('encounterId').cast(pl.String)
        ).join(
            encounters, on='encounterId', how="inner"
        ).with_columns(
            (((pl.col('utcChartTime') - pl.col('utcInTime')).dt.total_minutes())/60).alias('delta_inTime_hours')
        ).sort('encounterId', 'utcChartTime')
        )
        print(f"Nombre d'encounters distincts : {with_time_delta.unique('encounterId').shape[0]}/{encounters.shape[0]}")
        try :
            # Sauvegarde du dataset sous un format .parquet
            with_time_delta.write_parquet(f'../2.rawDataset/dynamic_features/{variable}.parquet')
            print(f'Sauvegarde {variable} : OK (shape : {with_time_delta.shape})')
        except Exception as e:
            print(e)
            continue
        
    except Exception as e:
        print(e)
        print(query)
        continue
    
    end_loop_time = time.time()    
    print(f'Elapsed time for {variable} : {round((end_loop_time - loop_time)/60, 0)}min (total exec : {round((end_loop_time-start)/60, 1)}min)')
    print(f'---------Extraction {variable} : END ---------')

In [ ]:
missing_encounters_folder = os.path.join(OUTPUT_FOLDER, 'missing_encounters')
for filename in os.listdir(OUTPUT_FOLDER):
    
    if filename.endswith(".parquet"):
        print(f'--------{filename}----------')
        # Charger le dataset .parquet
        feature_data = pl.read_parquet(os.path.join(OUTPUT_FOLDER, filename))
        
        # Récupérer les encounterId du dataset .parquet
        feature_encounters = feature_data.select("encounterId").unique()
        
        # Trouver les lignes du dataset encounters n'apparaissant pas dans la feature
        missing_encounters = encounters.join(feature_encounters, on="encounterId", how="anti")
        
        # Afficher le résultat
        print(f"Encounters manquants: {missing_encounters.shape[0]}/{encounters.unique('encounterId').shape[0]}")
        missing_filename = os.path.join(missing_encounters_folder, f"missing_{filename}")
        missing_encounters.write_parquet(missing_filename)